In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
#옵션설정
options = Options()
options.add_argument("disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--user_agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36")

In [ ]:
# 스크롤 로딩 -> 클래스 리스트에 담기 -> 링크로 들어가서 상세정보 더보기 누르기 -> 내용 리스트에 담기 -> 파일로 내보내기

In [ ]:
# 1. 스크롤 로딩

driver = webdriver.Chrome(options=options)
driver.get("https://www.wanted.co.kr/wdlist/507?country=kr&job_sort=job.popularity_order&years=-1&locations=all&selected=565&selected=559")

SCROLL_PAUSE = 1.0
MAX_TRIES   = 15   #최대반복횟수 수정

last_height = driver.execute_script("return document.body.scrollHeight")
tries = 0

while tries < MAX_TRIES:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE)

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    tries += 1

In [ ]:
#2. 링크로 들어가서 더보기 누르고 내용 수집

urls = []
result = []

items = driver.find_elements(By.CSS_SELECTOR, ".Card_Card__aaatv")
for i in items:
    url = i.find_element(By.CSS_SELECTOR, "div a").get_attribute("href")
    urls.append(url)

for i in urls:
    driver.get(i)
    btn_xpath = '/html/body/div[1]/main/div[1]/div/section/section/article[1]/div/button'
    driver.find_element(By.XPATH, btn_xpath).click()
    
    company_xpath = '/html/body/div[1]/main/div[1]/div/section/header/div/div[1]/a'
    company_name = driver.find_element(By.XPATH, company_xpath).text
    
    career_xpath = '/html/body/div[1]/main/div[1]/div/section/header/div/div[1]/span[4]'
    career = driver.find_element(By.XPATH, career_xpath).text
    
    info_xpath = '/html/body/div[1]/main/div[1]/div/section/section/article[1]/div/span/span'
    info = driver.find_element(By.XPATH, info_xpath).text.strip('\n')
    
    duty_xpath = '/html/body/div[1]/main/div[1]/div/section/section/article[1]/div/div[1]/span/span'
    duty = driver.find_element(By.XPATH, duty_xpath).text.strip('\n')
    
    require_xpath = '/html/body/div[1]/main/div[1]/div/section/section/article[1]/div/div[2]/span/span'
    require = driver.find_element(By.XPATH, require_xpath).text.strip('\n')
    
    header_xpath = '/html/body/div[1]/main/div[1]/div/section/section/article[1]/div/div[3]/h3'
    header = driver.find_element(By.XPATH, header_xpath).text
    
    if header == '우대사항':
        favor_xpath = '/html/body/div[1]/main/div[1]/div/section/section/article[1]/div/div[3]/span/span'
        favor = driver.find_element(By.XPATH, favor_xpath).text.strip('\n')
    else :
        favor = '없음'
    
    result.append({'company':company_name, 'career':career, 'info':info, 'duty':duty,
                   'require':require, 'favor':favor})
    
    driver.implicitly_wait(2)
 

driver.quit()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/main/div[1]/div/section/section/article[1]/div/button"}
  (Session info: chrome=141.0.7390.108); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x7ff63ac0e8e5+80021]
	GetHandleVerifier [0x0x7ff63ac0e940+80112]
	(No symbol) [0x0x7ff63a99060f]
	(No symbol) [0x0x7ff63a9e8854]
	(No symbol) [0x0x7ff63a9e8b1c]
	(No symbol) [0x0x7ff63aa3c927]
	(No symbol) [0x0x7ff63aa1126f]
	(No symbol) [0x0x7ff63aa3968a]
	(No symbol) [0x0x7ff63aa11003]
	(No symbol) [0x0x7ff63a9d95d1]
	(No symbol) [0x0x7ff63a9da3f3]
	GetHandleVerifier [0x0x7ff63aecdc7d+2960429]
	GetHandleVerifier [0x0x7ff63aec7f3a+2936554]
	GetHandleVerifier [0x0x7ff63aee8977+3070247]
	GetHandleVerifier [0x0x7ff63ac283ce+185214]
	GetHandleVerifier [0x0x7ff63ac2fe1f+216527]
	GetHandleVerifier [0

In [5]:
len(result)

325

In [6]:
#3. 파일로 내보내기

df = pd.DataFrame(result)
df.to_csv("check.csv", index=False, encoding='utf-8-sig')

프로젝트 회고

- Liked : 배우지 않은 부분도 스스로 찾아가며 해보니 시간은 오래걸려도 머리에는 더 오래 남게 됨.

- Learned : 프로젝트 목적을 더 디테일하게 고민해야 함을 깨달음. 코드를 짤 때 고민되는 지점이 생길 때 처음 프로젝트의 목적이 선택의 근거가 됨.

- Lacked : 계획할 때는 3개 사이트 데이터를 다 수집하려 했으나 실제로 해보니 1개 사이트만 완성할 수 있었음. 다음부터는 가지고 있는 자원을 잘 파악하고 계획을 세울 것. 코드 구성을 좀 더 효율적으로 해볼 것.